In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import os
import pandas as pd

# Path ke folder yang berisi file-file DataFrame
folder_path = '../comodity-price-prediction-penyisihan-arkavidia-9/Harga Bahan Pangan/train'

# Dictionary untuk menyimpan semua DataFrame dengan nama file sebagai key
df_dict = {}

# Loop melalui semua file dalam folder
for file_name in os.listdir(folder_path):
    # Pastikan file yang dibaca adalah file yang diinginkan (misalnya, CSV)
    if file_name.endswith('.csv'):
        # Buat path lengkap ke file
        file_path = os.path.join(folder_path, file_name)
        
        # Baca file dan simpan ke dictionary dengan nama file sebagai key
        df = pd.read_csv(file_path)
        df_dict[file_name] = df  # Gunakan nama file sebagai key


In [3]:
df_bawang_merah = df_dict['Bawang Merah.csv']
df_bawang_putih = df_dict['Bawang Putih Bonggol.csv']
df_beras_medium = df_dict['Beras Medium.csv']
df_beras_premium = df_dict['Beras Premium.csv']
df_cabai_merah_keriting = df_dict['Cabai Merah Keriting.csv']
df_cabai_rawit_merah = df_dict['Cabai Rawit Merah.csv']
df_daging_ayam = df_dict['Daging Ayam Ras.csv']
df_daging_sapi = df_dict['Daging Sapi Murni.csv']
df_gula = df_dict['Gula Konsumsi.csv']
df_minyak_goreng_curah = df_dict['Minyak Goreng Curah.csv']
df_minyak_goreng_kemasan = df_dict['Minyak Goreng Kemasan Sederhana.csv']
df_telur_ayam = df_dict['Telur Ayam Ras.csv']
df_tepung_terigu = df_dict['Tepung Terigu (Curah).csv']

In [4]:
import pandas as pd

# List semua dataframe
df_list = [
    df_bawang_merah, df_bawang_putih, df_beras_medium, df_beras_premium, df_cabai_merah_keriting,
    df_cabai_rawit_merah, df_daging_ayam, df_daging_sapi, df_gula, df_minyak_goreng_curah,
    df_minyak_goreng_kemasan, df_telur_ayam, df_tepung_terigu
]

# Loop setiap dataframe dan lakukan interpolasi
for i in range(len(df_list)):
    df = df_list[i]  # Ambil dataframe

    # Pisahkan kolom tanggal (asumsi ada kolom 'date' atau 'tanggal')
    date_cols = df.select_dtypes(include=['datetime64', 'object'])  # Deteksi kolom tanggal
    num_cols = df.select_dtypes(include=['number'])  # Ambil hanya kolom numerik

    # Interpolasi hanya untuk kolom numerik
    df_num_imputed = num_cols.interpolate(method='linear', limit_direction='both')

    # Gabungkan kembali kolom tanggal dengan hasil interpolasi
    df_list[i] = pd.concat([date_cols.reset_index(drop=True), df_num_imputed.reset_index(drop=True)], axis=1)


In [5]:
df_dict = {
    'Bawang Merah': df_list[0],
    'Bawang Putih Bonggol': df_list[1],
    'Beras Medium': df_list[2],
    'Beras Premium': df_list[3],
    'Cabai Merah Keriting': df_list[4],
    'Cabai Rawit Merah': df_list[5],
    'Daging Ayam Ras': df_list[6],
    'Daging Sapi Murni': df_list[7],
    'Gula Konsumsi': df_list[8],
    'Minyak Goreng Curah': df_list[9],
    'Minyak Goreng Kemasan Sederhana': df_list[10],
    'Telur Ayam Ras': df_list[11],
    'Tepung Terigu (Curah)': df_list[12]
}


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

# Set seed untuk reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dictionary untuk menyimpan hasil prediksi
all_predictions = []

# Rentang waktu prediksi
predict_start = '2024-10-01'
predict_end = '2024-12-31'
predict_periods = (pd.to_datetime(predict_end) - pd.to_datetime(predict_start)).days + 1

# Parameter LSTM
seq_length = 30
epochs = 100
batch_size = 16
learning_rate = 0.001

class LSTMPredictor(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=3, output_size=1):
        super(LSTMPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])

for komoditas, df in df_dict.items():
    print(f"\n=== Memproses {komoditas} ===")
    
    df['Date'] = pd.to_datetime(df['Date'])
    
    for provinsi in df.columns[1:]:
        print(f"  → Prediksi untuk {provinsi}...")
        
        df_prov = df[['Date', provinsi]].rename(columns={'Date': 'ds', provinsi: 'y'})
        
        # Normalisasi data
        scaler = MinMaxScaler()
        df_prov['y'] = scaler.fit_transform(df_prov[['y']])
        
        # Buat data sekuens
        X, y = [], []
        for i in range(len(df_prov) - seq_length):
            X.append(df_prov['y'].iloc[i:i+seq_length].values)
            y.append(df_prov['y'].iloc[i+seq_length])
        
        X, y = np.array(X), np.array(y)
        X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1).to(device)
        y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1).to(device)
        
        # Pisahkan training & testing
        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        # Bangun model LSTM
        model = LSTMPredictor().to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
        # Train model
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            output = model(X_train)
            loss = criterion(output, y_train)
            loss.backward()
            optimizer.step()
            
            if (epoch + 1) % 10 == 0:
                test_output = model(X_test)
                test_loss = criterion(test_output, y_test).item()
                print(f"Epoch {epoch+1}/{epochs}, Training Loss: {loss.item():.6f}, Testing Loss: {test_loss:.6f}")
        
        # Prediksi masa depan
        model.eval()
        last_sequence = X[-1].unsqueeze(0)  # Data terakhir sebagai input
        future_preds = []
        
        for _ in range(predict_periods):
            with torch.no_grad():
                next_pred = model(last_sequence).item()
            future_preds.append(next_pred)
            
            last_sequence = torch.cat([last_sequence[:, 1:, :], torch.tensor([[[next_pred]]], dtype=torch.float32).to(device)], dim=1)
        
        # Konversi kembali ke skala asli
        future_preds = scaler.inverse_transform(np.array(future_preds).reshape(-1, 1)).flatten()
        
        # Simpan hasil prediksi
        for i, pred in enumerate(future_preds):
            date = pd.to_datetime(predict_start) + pd.Timedelta(days=i)
            all_predictions.append({
                'id': f"{komoditas}/{provinsi}/{date.strftime('%Y-%m-%d')}",
                'price': pred
            })

# Simpan hasil prediksi ke CSV
submission_df = pd.DataFrame(all_predictions)
submission_df.to_csv('submission_lstm_pytorch_morelayer.csv', index=False)

print("\nSemua prediksi selesai dan disimpan di submission_lstm_pytorch.csv!")

# Ini paling bagus sekarang 
MAPE 0.080